In [1]:
from cell_mrcnn.utils import preproc_pipeline, get_cell_mrcnn_path_from_config_file, get_image_paths, convert_to_bit8
from cell_mrcnn.data import calculate_percentiles, preprocess, calc_avg_pixel_value, copy_annotated, CellTransformData
from skimage.io import imread
from PIL import Image
from os.path import join, isdir, split, dirname, realpath
from os import mkdir
import os
import glob
import argparse
from pathlib import Path
from PIL import Image
from glob import glob

In [2]:
# if u dont want to use the get_cell_mrcnn_path_from_config_file() u can just write your path/to/data folder below
data_dir = join(get_cell_mrcnn_path_from_config_file(), 'data')

In [3]:
ven_1210 = get_image_paths(join(data_dir,'20201210'), channels=('Venus'))

In [4]:
# these show the mean percentiles (for 8 bit conversion for the cutoff).
ven_1210_perc_df = calculate_percentiles(ven_1210)

In [5]:
ven_1210_perc_df.loc[ven_1210_perc_df.index[-10:],:]

,99.0000,99.9000,99.9900,99.9990,99.9999,100.0000
146,389,930,1872.53,3294,3294,3294
147,685,2627,4625,6248,6248,6248
148,1033,3472,4404,5124,5124,5124
149,1218,4421,7284,8185,8185,8185
min,0,0,10,173,173,173
max,1588.25,4646,8502,10841,10841,10841
mean,659.158,2042.64,3173.78,3724.93,3724.93,3724.93
median,627,2005.5,3193.99,3620.5,3620.5,3620.5
var,113372,961462,2.11575e+06,3.12314e+06,3.12314e+06,3.12314e+06
std,336.707,980.541,1454.56,1767.24,1767.24,1767.24


In [6]:
# the annotations go to the preproc_output_dir as well
preproc_output_dir = join(data_dir,'20201210/preprocessed/venus')

In [7]:
if not isdir(join(data_dir,'20201210/preprocessed')):
    mkdir(join(data_dir,'20201210/preprocessed'))
if not isdir(preproc_output_dir):
    mkdir(preproc_output_dir)
for i, impath in enumerate(ven_1210):
    im = imread(impath)
    im = convert_to_bit8(im, 4000)
    fname = split(impath)[1].split('.')[0]
    Image.fromarray(im).save(join(preproc_output_dir, fname+'.png'))

In [8]:
# once enough images have been annotated, copy the images and their annotations to a folder named the current datetime
dataset_dir = join(data_dir, 'annotated_datasets')
dataset_dir = copy_annotated(input_folder=preproc_output_dir, output_folder=dataset_dir)

In [10]:
ds = CellTransformData()
ds.load_cell(dataset_dir)
ds.prepare()
avg_pixel_vals = ds.split_to_trainval(dataset_dir)

Average pixel value(s) is(/are): [5.44183715]


In [9]:
impaths = glob(join(dataset_dir, '**/*.png'), recursive=True)
avg_pixel_vals = calc_avg_pixel_value(impaths)
print(avg_pixel_vals)

[5.441837151845296]
